<a href="https://colab.research.google.com/github/HSE-LAMBDA/DeepGenerativeModels/blob/master/seminars/seminar-3/mcmc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MCMC. Bayesian Inference
The bayesian inference problem is to infer posterior distribution given prior distribution and likelihood:
$$p(\theta|X)=\frac{p(X|\theta)p(\theta)}{\int_\theta p(X|\theta)p(\theta)d\theta}$$
In case of simple distributions bayesian inference can be performed analytically. However, in case of complicated distributions an explicit inference is intractable (because of denominator) and some more sophisticated techniques were designed. One of them is Markov Chain Monte Carlo (MCMC) sampling. It does not provides an option to perform a bayesian inference but helps to sample from the distribution (e.g., posterior) given that distribution up to a constant (denominator)

# Intro to Pyro
[Pyro](https://pyro.ai) is PyTorch-based framework for bayesian inference

## Toy example
Let $$p(weight|guess)=\mathcal{N}(guess,1)$$
$$p(measurement|weight,guess)=\mathcal{N}(weight,0.75)$$

Suppose we observe that the measurement of an object was 14 (kilos). What the most likelily weight and its confidence interval is?

Solution: let's sample from posterior distribution (weight given measurement) multiple times using MCMC and obtain an empirical estimations

In [1]:
# !pip install pyro-ppl
import pyro
import torch
from pyro.infer.mcmc import MCMC
from pyro.infer.mcmc.nuts import HMC
from pyro.infer import EmpiricalMarginal
import pyro.distributions as dist
import matplotlib.pyplot as plt


def model(guess): # probabilistic model
    weight = pyro.sample("weight", dist.Normal(guess, 1.0))
    measurement = pyro.sample("measurement", dist.Normal(weight, 0.75))
    return measurement

# %matplotlib inline
guess_prior = 10.
condition = pyro.condition(model, data={"measurement": torch.tensor(14.)})

hmc_kernel = HMC(condition, step_size=0.9, num_steps=4)
mcmc = MCMC(hmc_kernel, 
                 num_samples=1000, 
                 warmup_steps=50)
mcmc.run(guess_prior) # run MCMC sampling
samples = mcmc.get_samples()['weight']
print(f'Weight: {samples.mean().item()} +- {samples.std().item()}')

Sample: 100%|██████████| 1050/1050 [00:14, 73.65it/s, step size=2.72e-01, acc. prob=0.961]

Weight: 12.560986518859863 +- 0.5782628059387207


# Links
* [intro to Pyro and PPL](https://bookdown.org/robertness/causalml/docs/tutorial-on-deep-probabilitic-modeling-with-pyro.html)